## Search

In [8]:
from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults

search = DuckDuckGoSearchRun()
results = search.invoke("Obama's first name?")

In [5]:
search = DuckDuckGoSearchResults()
search.invoke("Obama's first name?")

"[snippet: Barack Obama—with his wife, Michelle—being sworn in as the 44th president of the United States, January 20, 2009. Key events in the life of Barack Obama. Barack Obama (born August 4, 1961, Honolulu, Hawaii, U.S.) is the 44th president of the United States (2009-17) and the first African American to hold the office., title: Barack Obama | Biography, Parents, Education, Presidency, Books ..., link: https://www.britannica.com/biography/Barack-Obama], [snippet: As the head of the government of the United States, the president is arguably the most powerful government official in the world. The president is elected to a four-year term via an electoral college system. Since the Twenty-second Amendment was adopted in 1951, the American presidency has been, title: list of presidents of the United States - Encyclopedia Britannica, link: https://www.britannica.com/topic/Presidents-of-the-United-States-1846696], [snippet: Barack Hussein Obama II (/ b ə ˈ r ɑː k h uː ˈ s eɪ n oʊ ˈ b ɑː m

In [18]:
from langchain import hub
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
import os
from dotenv import load_dotenv

load_dotenv()
hf_token = os.getenv("HF_TOKEN")

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    huggingfacehub_api_token=hf_token
)
llama = ChatHuggingFace(llm=llm)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\piamp\.cache\huggingface\token
Login successful


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Method 1 (create simple agent)

In [14]:
tools = [search]

from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llama, tools)

In [17]:
from langchain_core.messages import HumanMessage

agent.invoke({"messages": [HumanMessage(content="What is Obama's first name?")]})

d:\02-NextCloud\github\LangChain-Playground\.venv\lib\site-packages\huggingface_hub\inference\_generated\types\base.py:139: FutureWarning: Accessing 'ChatCompletionOutputToolCall' values through dict is deprecated and will be removed from version '0.25'. Use dataclass attributes instead.
  warnings.warn(


{'messages': [HumanMessage(content="What is Obama's first name?", id='5fefb2cc-edc4-4b84-8e81-5ffb18ef13c8'),
  AIMessage(content='', additional_kwargs={'tool_calls': [ChatCompletionOutputToolCall(function=ChatCompletionOutputFunctionDefinition(arguments={'query': "What is Obama's first name?"}, name='duckduckgo_search', description=None), id='0', type='function')]}, response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=29, prompt_tokens=175, total_tokens=204), 'model': '', 'finish_reason': 'eos_token'}, id='run-1f7c2b0f-c2b0-4024-bb6d-4178606521b9-0', invalid_tool_calls=[{'name': 'duckduckgo_search', 'args': '{"query": "What is Obama"s first name?"}', 'id': '0', 'error': None, 'type': 'invalid_tool_call'}])]}

## Method 2

In [7]:
prompt = hub.pull("rlm/rag-prompt")

In [9]:
chain = prompt | llama
chain.invoke({
    "context": results,
    "question": "Obama's first name?"
})

AIMessage(content='Barack.', response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=4, prompt_tokens=443, total_tokens=447), 'model': '', 'finish_reason': 'eos_token'}, id='run-3cad16d5-406e-45ae-a723-0032a1c8fcb6-0')